<center><h1>TinyML & Edge Impulse</h1></center>

<h2>Overview</h2>

<ul>
<li>Challenges of Centralized Machine Learning</li>
<li>What is TinyML?</li>
<li>TinyML Application Design Flow</li>
<li>TinyML Software Suites</li>
<li>TinyML Software Suites</li>
    <li>What is Edge Impulse?</li>
    <li>Case Study: Build a Machine Learning Model to Switch On/Off Devices using Voice Commands</li>      
    </ul>

<h1>Food for Thought!!</h1>

Let's watch this <a href="https://www.youtube.com/watch?v=ySQTcCOziXs"> video</a> here: https://www.youtube.com/watch?v=ySQTcCOziXs

<h2>Moving ML from the Cloud to the Edge</h2>

<ul>
    <li>Most of data going to cloud is from IoT devices</li>
    <li>Around 328.77 million terabytes of data are generated each day. <a href="https://explodingtopics.com/blog/data-generated-per-day">Check this reference for more information</a></li>
    <li>Less than half of structured data is actively used in decision making</li>
    <li>Less than 1% of unstructured data is analyzed or used at all
    <li>ML at the edge can help use that data locally to make decision
    <li>ML + low-power, low-cost devices + low latency = <b>NEW OPPORTUNITIES</b>
</ul>

<h1>What is TinyML?</h1>

<ul>
<li><b>Tiny Machine Learning (TinyML)</b> is a fast-growing field of machine learning technologies and applications including algorithms, hardware, and software capable of performing on-device sensor data analytics at extremely low power consumption, typically in the mW range and below.</li>
    <li>TinyML enables a variety of always-on ML use-cases on battery-operated
devices</li>
</ul>

<h2>TinyML Domain Interaction</h2>

<center><figure><img src="https://www.andrew.cmu.edu/user/mfarag/static/tinyml_domains.png"/><figcaption>TinyML lies at the Intersection</figcaption></figure></center>

<h2>TinyML Application Design Flow</h2>

<center><figure><img src="https://www.andrew.cmu.edu/user/mfarag/static/tinyml_design_flow.png"/><figcaption>TinyML App Design Flow</figcaption></figure></center>

<h1>TinyML Software Suites</h1>

<ul>
<li><b>TensorFlow Lite:</b> An open-source framework for deploying machine learning models on edge devices.</li>
<li><b>Arm Cortex-M microcontroller-based machine learning:</b> A suite of tools and libraries for implementing machine learning algorithms on Arm Cortex-M microcontrollers.</li>
<li><b>Zephyr Project:</b> An open-source real-time operating system for building and deploying machine learning algorithms on IoT devices.</li>
    </ul>

<ul>
<li><b>Arduino and Arduino Create:</b> An open-source platform for building and programming electronic devices, including those with machine learning capabilities. Create is an online platform for creating and deploying IoT applications, including those using TinyML algorithms.</li>
    <li><b>Edge Impulse:</b> A platform for developing and deploying machine learning algorithms on edge devices.</li>
<li><b>OpenMV:</b> An open-source hardware and software platform for developing vision-based applications on microcontrollers.</li>
<li><b>PYNQ:</b> This is an open-source framework for developing ML algorithms on Xilinx Field-Programmable Gate Arrays (FPGAs).</li>
    <li><b>MicroPython:</b> An open-source Python interpreter that can be used to develop and run TinyML algorithms on microcontrollers and other low-power devices.</li>
</ul>

In this course, we will use <b>Edge Impulse</b> for the following reasons:
- Ease of use with customized models.
- Requires minimal setup on your machine.
- Offers a Browser-based solution that is compatible with every operating system.
- Supports wide number of development boards.
- Scalable and integrates well with other tools.
- Offers low-latency and energy efficient solutions.


<h1>Edge Impulse Process Flow</h1>

Edge Impulse reduces the number of tasks required to design ML application to 5 UI-led tasks:
<ul>
    <li>Collect Data</li>
    <li>Clean Data and Generate Features</li>
    <li>Train Your Model and Test It</li>
    <li>Find the Best Model for Your Target Device</li>
    <li>Deploy the Model</li>
</ul>

<h1>Case Study: Build a Machine Learning Model to Turn On/Off Devices using Voice Commands</h1>

<h1>1. Create Edge Impulse Account</h1>

Before we start using Edge Impulse, you need to create a community edition account using your @andrew.cmu.edu email. Use this URL to sign up: <a href="https://studio.edgeimpulse.com/signup">https://studio.edgeimpulse.com/signup</a>

<h1>2. Create the Dataset</h1>

The dataset that we will use contains Speech commands for more than 60,000 instances of one-second-long utterances of 30 different spoken keywords such as "yes", "no", "up", "down", "left", "right", "on", "off", and many more.  These commands are stored as digital audio files. The audio files are sampled at 16 kilohertz. You may refer to the paper
<i>"Speech Commands: A Dataset for Limited-Vocabulary Speech Recognition" by Pete Warden to know more about the dataset.</i>

In [1]:
!pip install -U -q tensorflow tensorflow_datasets
#!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

<h2>2.1 Import the necessary packages</h2>

In [2]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from numpy import random
import shutil, errno

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models

<h2>2.2 Download the speech commands dataset</h2>
A new directory named data will be created, which will contain subdirectories corresponding to all 30 keywords. Each subdirectory will contain the corresponding audio files in WAV format.

In [3]:
DATASET_PATH = 'data/'

data_dir = pathlib.Path(DATASET_PATH)
if not data_dir.exists():
    tf.keras.utils.get_file(
      'speech_commands.zip',
      origin='http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz',
      extract=True,
      cache_dir='.', cache_subdir='data')

<b>We want to develop a Turn on/Off Switch that includes keyword spotting algorithm so it can detect the target keywords "on" and "off". However, creating a binary classifier may not be sufficient, as the input audio could contain human speech with other words or may have no background audio at all</b>

<h2>2.3 Our Multi-class Classifier</h2>

We will create a multi-class classifier to detect four different class labels: "on", "off", "others", and "silent". So, we
need to form a new dataset from the original Speech Commands dataset containing audio instances corresponding to the four target classes we are going to detect. We will create a new empty directory, mydatset, under the parent directory data and we will populate the labels accordingly.

<h2>2.4 Get different class labels</h2>

In [4]:
commands = np.array(tf.io.gfile.listdir(str('data')))
commands = commands[commands != 'README.md']
print('Commands:', commands)

Commands: ['.DS_Store' 'backward' 'bed' 'bird' 'cat' 'dog' 'down' 'eight' 'five'
 'follow' 'forward' 'four' 'go' 'happy' 'house' 'learn' 'left' 'LICENSE'
 'marvin' 'mydataset' 'nine' 'no' 'off' 'on' 'one' 'right' 'seven'
 'sheila' 'six' 'speech_commands.zip' 'stop' 'testing_list.txt' 'three'
 'tree' 'two' 'up' 'validation_list.txt' 'visual' 'wow' 'yes' 'zero'
 '_background_noise_']


<h2>2.5 Form the directory for string on, off, others and silent data</h2>

In [6]:
try:
    os.mkdir('data/mydataset')
except Exception as e:
    print("Error creating folder. Error details {}".format(e))

In [7]:

def copy_data(src, dst):
    try:
        shutil.copytree(src, dst)
    except OSError as exc: # python >2.5
        if exc.errno in (errno.ENOTDIR, errno.EINVAL):
            shutil.copy(src, dst)
        else: raise


In [8]:
copy_data('data/on','data/mydataset/on')
copy_data('data/off','data/mydataset/off')

In [9]:
# You may use the following code instead of calling the copy_data() function above if you are not using windows
#!cp -r 'data/on' 'data/mydataset'
#!cp -r 'data/off' 'data/mydataset'

In [10]:
os.mkdir('data/mydataset/silent')
os.mkdir('data/mydataset/others')

<h3>2.5.1 Populate "Others" label data</h3>

In [11]:
other_labels = ["yes", "no", "up", "down", "left", "right","bed", "bird", "cat", "dog", "happy", "house", "marvin", "sheila", "tree", "wow"]
sample_per_label = 150


for label in other_labels:
    path = 'data/'+label
    f = os.listdir(path)
    num_files = len(f)
    file_indx = np.arange(num_files)
    random.shuffle(file_indx)
    for i in range(sample_per_label):
        index = file_indx[i]
        file_name = f[index]
        source = path + '/' + file_name
        destination = 'data/mydataset/others/' + file_name
        # copy only files
        if os.path.isfile(source):
            shutil.copy(source, destination)

<h3>2.5.2 Create "Silent" label data</h3>

In [12]:
import scipy
from scipy.io.wavfile import write

fs = 16000
num_files = 2400

for i in range(num_files):
    sample = np.zeros(fs)
    filename = str(i*100)+'silent.wav'
    sample = sample + 0.01*i*random.randn(fs)
    scipy.io.wavfile.write('data/mydataset/silent/'+filename, fs, sample.astype(np.int16))

<h2>2.6 Upload the Data to Edge Impulse</h2>
Use the data inside <b>mydataset</b> folder. Follow the Demo Carefully and watch the Lecture recording if needed. It's OK if some of the silent data were not uploaded due to identical hash values. You may also try to create the silent files yourself and upload them.

<h1>3. Add a Processing Block to Generate Features for ML Model</h1>

We will use <b>Mel Frequency Cepstral Coefficient (MFCC)</b> algorithm for training our model. MFCC is an audio feature extraction algorithm commonly used in speech processing and speaker recognition applications. In MFCC, the frequency bands of the audio are mapped on the Mel scale, which closely approximates the human auditory system.

MFCC is considered as a better feature extraction technique compared
to the spectrogram-based approach in speech processing applications. The MFCC
components are calculated through the following steps:
1. Break the audio signals into small windows.
2. Calculate the spectrum of the windows via Short-Time Fourier Transform.
3. Map the spectrum power into equivalent Mel scale.
4. Calculate the logarithm of the power components at the Mel frequencies.
5. Calculate the discrete cosine transform of the logarithm powers.
6. The resulting amplitudes represent the MFCC values.

<h1>4. Add a Learning Block to Train Your Model and Test It</h1>
Keep in mind, the device we will target for this example is: <b>Arduino Nano 33 BLE Sense (Cortex-M4F 64MHz)</b>

<h1>5. Add a new Device as Your Smartphone using QR Code and Use Your Phone to Conduct Live Classification</h1>

<h1>Is this the best model for the Target Hardware?</h1>
We will answer this question next lecture!